<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-3/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install
# !pip install --upgrade google-cloud-bigquery==2.30.1
!pip install --upgrade google-cloud-bigquery
#!pip install stargazer
#!pip install git+https://github.com/statsmodels/statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/4 Vaccination and Election Results/Data'
os.chdir(path)

project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# from stargazer.stargazer import Stargazer

from zipfile import ZipFile

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

auth.authenticate_user()
project_id = "phd-dissertation-343113"
!gcloud config set project {project_id}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated property [core/project].


In [ ]:
# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query
query = """
WITH base_query AS (
SELECT 
  id_paciente,
  id_municipio_endereco_paciente,
  id_municipio_estabelecimento,
  cep_endereco_paciente,
  data_aplicacao_vacina,
  dose_vacina,
  ROW_NUMBER() OVER (PARTITION BY id_paciente ORDER BY data_aplicacao_vacina) AS dose_number
  FROM `basedosdados.br_ms_vacinacao_covid19.microdados`
), pivot_query_applied AS (
  SELECT 
    id_municipio_estabelecimento as id_municipio,
    COUNTIF(dose_number = 1 AND dose_vacina IS NOT NULL) AS dose1_applied,
    COUNTIF(dose_number = 2 AND dose_vacina IS NOT NULL) AS dose2_applied,
    COUNTIF(dose_number = 3 AND dose_vacina IS NOT NULL) AS dose3_applied,
    COUNTIF(dose_number = 4 AND dose_vacina IS NOT NULL) AS dose4_applied
  FROM base_query
  WHERE dose_vacina IS NOT NULL
  GROUP BY id_municipio_estabelecimento
), pivot_query_received AS (
  SELECT 
    id_municipio_endereco_paciente as id_municipio,
    COUNTIF(dose_number = 1 AND dose_vacina IS NOT NULL) AS dose1_received,
    COUNTIF(dose_number = 2 AND dose_vacina IS NOT NULL) AS dose2_received,
    COUNTIF(dose_number = 3 AND dose_vacina IS NOT NULL) AS dose3_received,
    COUNTIF(dose_number = 4 AND dose_vacina IS NOT NULL) AS dose4_received
  FROM base_query
  WHERE dose_vacina IS NOT NULL
  GROUP BY id_municipio_endereco_paciente
), population AS (
  SELECT 
  m.id_municipio as id_municipio, 
  m.nome as name, 
  m.sigla_uf as uf, 
  m.nome_regiao as region, 
  p.populacao as population
FROM 
  `basedosdados.br_bd_diretorios_brasil.municipio` AS m
JOIN 
  `basedosdados.br_ibge_populacao.municipio` AS p
ON 
  m.id_municipio = p.id_municipio
JOIN (
  SELECT 
    id_municipio, 
    MAX(ano) AS max_ano 
  FROM 
    `basedosdados.br_ibge_populacao.municipio` 
  GROUP BY 
    id_municipio
) AS max_ano
ON 
  p.id_municipio = max_ano.id_municipio 
  AND p.ano = max_ano.max_ano
)
SELECT
  COALESCE(pivot_query_applied.id_municipio, pivot_query_received.id_municipio) AS id_municipio,
  IFNULL(dose1_applied, 0) AS dose1_applied,
  IFNULL(dose2_applied, 0) AS dose2_applied,
  IFNULL(dose3_applied, 0) AS dose3_applied,
  IFNULL(dose4_applied, 0) AS dose4_applied,
  IFNULL(dose1_received, 0) AS dose1_received,
  IFNULL(dose2_received, 0) AS dose2_received,
  IFNULL(dose3_received, 0) AS dose3_received,
  IFNULL(dose4_received, 0) AS dose4_received
FROM pivot_query_applied
FULL OUTER JOIN pivot_query_received
ON pivot_query_applied.id_municipio = pivot_query_received.id_municipio;
"""

# Execute the query and store the results in a dataframe
df = client.query(query).to_dataframe()
df.head(5)

,id_municipio,dose1_applied,dose2_applied,dose3_applied,dose4_applied,dose1_received,dose2_received,dose3_received,dose4_received
0,3302700,131000,95267,14048,138,129682,97964,17451,217
1,3545803,153833,130094,29872,2618,153528,130440,29660,2524
2,2904605,54352,33207,2181,8,58953,37095,2948,68
3,3152105,51726,43838,7219,42,53374,45339,7638,72
4,1502202,50659,34613,2971,50,54297,37331,3303,60


In [ ]:
# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query
query = """
SELECT 
  m.id_municipio,
  m.id_municipio_tse, 
  m.nome, 
  m.sigla_uf, 
  m.nome_regiao, 
  p.populacao
FROM 
  `basedosdados.br_bd_diretorios_brasil.municipio` AS m
JOIN 
  `basedosdados.br_ibge_populacao.municipio` AS p
ON 
  m.id_municipio = p.id_municipio
JOIN (
  SELECT 
    id_municipio, 
    MAX(ano) AS max_ano 
  FROM 
    `basedosdados.br_ibge_populacao.municipio` 
  GROUP BY 
    id_municipio
) AS max_ano
ON 
  p.id_municipio = max_ano.id_municipio 
  AND p.ano = max_ano.max_ano
"""

# Execute the query and store the results in a dataframe
df2 = client.query(query).to_dataframe()
df2.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516
1,1100023,78,Ariquemes,RO,Norte,111148
2,1100031,450,Cabixi,RO,Norte,5067
3,1100049,94,Cacoal,RO,Norte,86416
4,1100056,272,Cerejeiras,RO,Norte,16088


In [ ]:
df = df2.merge(df, on="id_municipio")
df.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao,dose1_applied,dose2_applied,dose3_applied,dose4_applied,dose1_received,dose2_received,dose3_received,dose4_received
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,17525,14748,2325,6,19517,15960,2513,13
1,1100023,78,Ariquemes,RO,Norte,111148,73445,58687,12299,355,80971,62918,12986,359
2,1100031,450,Cabixi,RO,Norte,5067,4466,3736,580,45,4734,3823,584,49
3,1100049,94,Cacoal,RO,Norte,86416,73031,54926,9317,118,71280,55122,9616,145
4,1100056,272,Cerejeiras,RO,Norte,16088,12576,10441,2344,28,13389,10732,2382,33


In [ ]:
df['doses_applied'] = df['dose1_applied'] + df['dose2_applied'] + df['dose3_applied'] + df['dose4_applied']
df['doses_received'] = df['dose1_received'] + df['dose2_received'] + df['dose3_received'] + df['dose4_received']
df.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao,dose1_applied,dose2_applied,dose3_applied,dose4_applied,dose1_received,dose2_received,dose3_received,dose4_received,doses_applied,doses_received
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,17525,14748,2325,6,19517,15960,2513,13,34604,38003
1,1100023,78,Ariquemes,RO,Norte,111148,73445,58687,12299,355,80971,62918,12986,359,144786,157234
2,1100031,450,Cabixi,RO,Norte,5067,4466,3736,580,45,4734,3823,584,49,8827,9190
3,1100049,94,Cacoal,RO,Norte,86416,73031,54926,9317,118,71280,55122,9616,145,137392,136163
4,1100056,272,Cerejeiras,RO,Norte,16088,12576,10441,2344,28,13389,10732,2382,33,25389,26536


In [ ]:
df['doses_applied'].sum(), df['doses_received'].sum()

(323590809, 323590809)

In [ ]:
# global variables
LOCAL = 'BRASIL' # UF ou BRASIL
file_results_2022 = os.path.join(path, 'votacao_candidato_munzona_2022.zip')
file_results_2018 = os.path.join(path, 'votacao_candidato_munzona_2018.zip')

In [ ]:
# Presidential elections runoff results

with ZipFile(file_results_2018) as z:
  with z.open(f'votacao_candidato_munzona_2018_{LOCAL}.csv') as f:
    results_2018 = pd.read_csv(f,sep=';',encoding='ISO-8859-1', decimal=','
    ).rename(columns = {'CD_MUNICIPIO' : 'id_municipio_tse'})

results_2018 = results_2018[(results_2018['DS_CARGO'] == 'Presidente') & (results_2018['NR_TURNO'] == 2 )]

In [ ]:
with ZipFile(file_results_2022) as z:
  with z.open(f'votacao_candidato_munzona_2022_{LOCAL}.csv') as f:
    results_2022 = pd.read_csv(f,sep=';',encoding='ISO-8859-1', decimal=','
    ).rename(columns = {'CD_MUNICIPIO' : 'id_municipio_tse'})

results_2022 = results_2022[(results_2022['DS_CARGO'] == 'Presidente') & (results_2022['NR_TURNO'] == 2 )]

In [ ]:
results_2018['id_municipio_tse'] = results_2018['id_municipio_tse'].astype(str)
results_2022['id_municipio_tse'] = results_2022['id_municipio_tse'].astype(str)

<ipython-input-52-d825b674edb0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_2022['id_municipio_tse'] = results_2022['id_municipio_tse'].astype(str)


In [ ]:
# grouping results by city

results_2018 = results_2018.groupby(['id_municipio_tse','NM_URNA_CANDIDATO']).agg(
        { 'QT_VOTOS_NOMINAIS':'sum'})

results_2018 = results_2018.pivot_table('QT_VOTOS_NOMINAIS', "id_municipio_tse", "NM_URNA_CANDIDATO")

In [ ]:
results_2022 = results_2022.groupby(['id_municipio_tse','NM_URNA_CANDIDATO']).agg(
        { 'QT_VOTOS_NOMINAIS':'sum'})

results_2022 = results_2022.pivot_table('QT_VOTOS_NOMINAIS', "id_municipio_tse", "NM_URNA_CANDIDATO")

In [ ]:
results_2022.head(5)

NM_URNA_CANDIDATO,JAIR BOLSONARO,LULA
id_municipio_tse,,
10006,864,4864
10014,640,3165
10022,636,2584
10030,3351,8576
10049,2437,5723


In [ ]:
results_2018.head(5)

NM_URNA_CANDIDATO,FERNANDO HADDAD,JAIR BOLSONARO
id_municipio_tse,,
10006,4331.0,705.0
10014,2963.0,623.0
10022,2687.0,498.0
10030,7942.0,2799.0
10049,5007.0,1605.0


In [ ]:
# calculating the voting share
results_2018['Bolsonaro_share_2018'] = (
    results_2018['JAIR BOLSONARO'] / (results_2018['FERNANDO HADDAD'] + results_2018['JAIR BOLSONARO'])
    )

results_2022['Bolsonaro_share_2022'] = (
    results_2022['JAIR BOLSONARO'] / (results_2022['LULA'] + results_2022['JAIR BOLSONARO'])
    )

df['id_municipio_tse'].astype('int64')

df = pd.merge(df, results_2018['Bolsonaro_share_2018'], on='id_municipio_tse')
df = df.merge(results_2022['Bolsonaro_share_2022'], on='id_municipio_tse')

In [ ]:
df['id_municipio_tse'] = df['id_municipio_tse'].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_municipio          5570 non-null   object 
 1   id_municipio_tse      5570 non-null   object 
 2   nome                  5570 non-null   object 
 3   sigla_uf              5570 non-null   object 
 4   nome_regiao           5570 non-null   object 
 5   populacao             5570 non-null   Int64  
 6   dose1_applied         5570 non-null   Int64  
 7   dose2_applied         5570 non-null   Int64  
 8   dose3_applied         5570 non-null   Int64  
 9   dose4_applied         5570 non-null   Int64  
 10  dose1_received        5570 non-null   Int64  
 11  dose2_received        5570 non-null   Int64  
 12  dose3_received        5570 non-null   Int64  
 13  dose4_received        5570 non-null   Int64  
 14  doses_applied         5570 non-null   Int64  
 15  doses_received       

In [ ]:
# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query
query = """
WITH deputado AS(
  SELECT id_municipio,
  SUM(CASE WHEN ano = 2018 AND sigla_partido IN ('PSL', 'PRTB') THEN votos ELSE 0 END) AS votos_PSL_2018,
  SUM(CASE WHEN ano = 2018 THEN votos ELSE 0 END) AS votos_tot_2018,
  SUM(CASE WHEN ano = 2022 AND sigla_partido IN ('PL', 'PP', 'REPUBLICANOS') THEN votos ELSE 0 END) AS votos_PL_2022,
  SUM(CASE WHEN ano = 2022 THEN votos ELSE 0 END) AS votos_tot_2022
  FROM `basedosdados.br_tse_eleicoes.resultados_candidato_secao`
  WHERE turno = 1 AND cargo = 'deputado federal'
  GROUP BY id_municipio
),
presidente AS(
  SELECT id_municipio,
  SUM(CASE WHEN ano = 2018 AND sigla_partido IN ('PSOL','PSTU') THEN votos ELSE 0 END) AS far_left_2018,
  SUM(CASE WHEN ano = 2018 AND sigla_partido IN ('PATRIOTA','NOVO') THEN votos ELSE 0 END) AS far_right_2018,
  SUM(CASE WHEN ano = 2022 AND sigla_partido IN ('UP', 'PCB', 'PSTU') THEN votos ELSE 0 END) AS far_left_2022,
  SUM(CASE WHEN ano = 2022 AND sigla_partido IN ('PTB', 'NOVO') THEN votos ELSE 0 END) AS far_right_2022
  FROM `basedosdados.br_tse_eleicoes.resultados_candidato_municipio`
  WHERE turno = 1 AND cargo = 'presidente'
  GROUP BY id_municipio
),
combined AS (
  SELECT COALESCE(deputado.id_municipio, presidente.id_municipio) AS id_municipio,
  presidente.far_left_2018 AS far_left_2018,
  presidente.far_left_2022 AS far_left_2022,
  presidente.far_right_2018 AS far_right_2018,
  presidente.far_right_2022 AS far_right_2022,
  deputado.votos_psl_2018 AS votos_psl_2018,
  deputado.votos_pl_2022 AS votos_pl_2022,
  deputado.votos_tot_2018 AS votos_tot_2018,
  deputado.votos_tot_2022 AS votos_tot_2022
  FROM deputado
  FULL OUTER JOIN presidente ON deputado.id_municipio = presidente.id_municipio
)
SELECT *
FROM combined;
"""

# Execute the query and store the results in a dataframe
df2 = client.query(query).to_dataframe()
df2.head(5)

,id_municipio,far_left_2018,far_left_2022,far_right_2018,far_right_2022,votos_psl_2018,votos_pl_2022,votos_tot_2018,votos_tot_2022
0,2502409,10,2,12,6,137,3285,4904,5372
1,2100832,20,3,108,9,264,4352,7969,9733
2,2912707,30,6,155,8,140,2279,8476,9294
3,3510906,11,1,71,9,145,376,1377,1722
4,4119103,19,2,192,33,773,853,6806,7640


In [ ]:
df2.columns

Index(['id_municipio', 'far_left_2018', 'far_left_2022', 'far_right_2018',
       'far_right_2022', 'votos_psl_2018', 'votos_pl_2022', 'votos_tot_2018',
       'votos_tot_2022'],
      dtype='object')

In [ ]:
columns2018 = ['far_left_2018', 'far_right_2018', 'votos_psl_2018']
columns2022 = ['far_left_2022', 'far_right_2022', 'votos_pl_2022']

for column in columns2018:
  df2[column] = df2[column] / df2['votos_tot_2018']

for column in columns2022:
  df2[column] = df2[column] / df2['votos_tot_2022']

df2.head(5)

,id_municipio,far_left_2018,far_left_2022,far_right_2018,far_right_2022,votos_psl_2018,votos_pl_2022,votos_tot_2018,votos_tot_2022
0,2502409,0.002039,0.000372,0.002447,0.001117,0.027936,0.611504,4904,5372
1,2100832,0.00251,0.000308,0.013553,0.000925,0.033128,0.447139,7969,9733
2,2912707,0.003539,0.000646,0.018287,0.000861,0.016517,0.245212,8476,9294
3,3510906,0.007988,0.000581,0.051561,0.005226,0.105301,0.218351,1377,1722
4,4119103,0.002792,0.000262,0.02821,0.004319,0.113576,0.111649,6806,7640


In [ ]:
columns2018 + columns2022

['far_left_2018',
 'far_right_2018',
 'votos_psl_2018',
 'far_left_2022',
 'far_right_2022',
 'votos_pl_2022']

In [ ]:
df2 = df2[['id_municipio','far_left_2018','far_right_2018','votos_psl_2018','far_left_2022','far_right_2022','votos_pl_2022']]
df2.head(5)

,id_municipio,far_left_2018,far_right_2018,votos_psl_2018,far_left_2022,far_right_2022,votos_pl_2022
0,2502409,0.002039,0.002447,0.027936,0.000372,0.001117,0.611504
1,2100832,0.00251,0.013553,0.033128,0.000308,0.000925,0.447139
2,2912707,0.003539,0.018287,0.016517,0.000646,0.000861,0.245212
3,3510906,0.007988,0.051561,0.105301,0.000581,0.005226,0.218351
4,4119103,0.002792,0.02821,0.113576,0.000262,0.004319,0.111649


In [ ]:
df = df.merge(df2, on='id_municipio')
df.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao,dose1_applied,dose2_applied,dose3_applied,dose4_applied,...,doses_applied,doses_received,Bolsonaro_share_2018,Bolsonaro_share_2022,far_left_2018,far_right_2018,votos_psl_2018,far_left_2022,far_right_2022,votos_pl_2022
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,17525,14748,2325,6,...,34604,38003,0.693741,0.711590,0.007669,0.022286,0.089687,0.000478,0.001673,0.340554
1,1100023,78,Ariquemes,RO,Norte,111148,73445,58687,12299,355,...,144786,157234,0.808128,0.791467,0.007493,0.029022,0.110335,0.000818,0.003756,0.109731
2,1100031,450,Cabixi,RO,Norte,5067,4466,3736,580,45,...,8827,9190,0.658672,0.684990,0.006693,0.02008,0.130187,0.000306,0.003671,0.303457
3,1100049,94,Cacoal,RO,Norte,86416,73031,54926,9317,118,...,137392,136163,0.780883,0.753317,0.007041,0.037783,0.113584,0.000638,0.004402,0.184936
4,1100056,272,Cerejeiras,RO,Norte,16088,12576,10441,2344,28,...,25389,26536,0.814381,0.797846,0.005307,0.014151,0.251179,0.000661,0.001984,0.194444


In [ ]:
# Define the query
query = """
SELECT id_municipio,
cobertura_total as GeneralVaccination,
cobertura_bcg as BCG_coverage,
cobertura_poliomielite as Polio_coverage
FROM `basedosdados.br_ms_imunizacoes.municipio`
WHERE ano = 2020
"""

# Execute the query and store the results in a dataframe
df2 = client.query(query).to_dataframe()
df2.head(5)

,id_municipio,GeneralVaccination,BCG_coverage,Polio_coverage
0,5300108,76.00,90.56,81.46
1,2300101,46.95,15.50,63.57
2,2300150,82.69,74.27,102.43
3,2300200,56.88,17.53,74.34
4,2300309,65.01,75.13,82.24


In [ ]:
df = df.merge(df2, on='id_municipio')
df.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao,dose1_applied,dose2_applied,dose3_applied,dose4_applied,...,Bolsonaro_share_2022,far_left_2018,far_right_2018,votos_psl_2018,far_left_2022,far_right_2022,votos_pl_2022,GeneralVaccination,BCG_coverage,Polio_coverage
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,17525,14748,2325,6,...,0.711590,0.007669,0.022286,0.089687,0.000478,0.001673,0.340554,84.66,82.20,94.92
1,1100023,78,Ariquemes,RO,Norte,111148,73445,58687,12299,355,...,0.791467,0.007493,0.029022,0.110335,0.000818,0.003756,0.109731,81.06,81.38,86.60
2,1100031,450,Cabixi,RO,Norte,5067,4466,3736,580,45,...,0.684990,0.006693,0.02008,0.130187,0.000306,0.003671,0.303457,95.99,56.72,122.39
3,1100049,94,Cacoal,RO,Norte,86416,73031,54926,9317,118,...,0.753317,0.007041,0.037783,0.113584,0.000638,0.004402,0.184936,83.54,85.74,84.78
4,1100056,272,Cerejeiras,RO,Norte,16088,12576,10441,2344,28,...,0.797846,0.005307,0.014151,0.251179,0.000661,0.001984,0.194444,95.35,84.12,103.97


In [ ]:
file_list = ['elderly.csv', 'rural_pop.csv', 'skin_color.csv', 'population_density.csv', 'religion.csv', 'years_educ.csv', 'house_income.csv']

df_list = []
for file in file_list:
    df2 = pd.read_csv(file)
    df_list.append(df2)

# Merge all DataFrames in df_list by the column "id_municipio"
df_merged = df_list[0] # start with the first DataFrame in the list
for df2 in df_list[1:]:
    df_merged = pd.merge(df_merged, df2, on='id_municipio')

# Display the merged DataFrame
df_merged.head(5)

<ipython-input-67-fde1a7b61543>:11: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Município_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged = pd.merge(df_merged, df2, on='id_municipio')


,id_municipio,Município_x,Situação do domicílio_x,Grupo de idade_x,elderly(>70yo),Município_y,Situação do domicílio_y,Grupo de idade_y,rural_pop,Município_x,...,<1,1-3,4-7,8-10,11-14,>15,NA,Município,Destino do lixo,house_income_BRL
0,1100015,Alta Floresta D'Oeste (RO),Total,70 anos ou mais,3.76,Alta Floresta D'Oeste (RO),Rural,Total,42.73,Alta Floresta D'Oeste (RO),...,13.79,27.30,40.14,10.13,6.52,0.74,1.37,Alta Floresta D'Oeste (RO),Total,1310.78
1,1100023,Ariquemes (RO),Total,70 anos ou mais,2.60,Ariquemes (RO),Rural,Total,15.30,Ariquemes (RO),...,11.32,20.09,40.11,15.57,10.28,1.81,0.82,Ariquemes (RO),Total,1849.30
2,1100031,Cabixi (RO),Total,70 anos ou mais,4.13,Cabixi (RO),Rural,Total,57.34,Cabixi (RO),...,14.07,24.92,43.54,9.87,5.25,0.29,2.06,Cabixi (RO),Total,1232.03
3,1100049,Cacoal (RO),Total,70 anos ou mais,3.48,Cacoal (RO),Rural,Total,21.19,Cacoal (RO),...,11.51,19.50,39.99,13.75,11.12,1.97,2.16,Cacoal (RO),Total,1853.64
4,1100056,Cerejeiras (RO),Total,70 anos ou mais,4.18,Cerejeiras (RO),Rural,Total,15.33,Cerejeiras (RO),...,11.52,21.62,43.38,12.36,8.83,0.64,1.65,Cerejeiras (RO),Total,1573.62


In [ ]:
df_merged.columns

Index(['id_municipio', 'Município_x', 'Situação do domicílio_x',
       'Grupo de idade_x', 'elderly(>70yo)', 'Município_y',
       'Situação do domicílio_y', 'Grupo de idade_y', 'rural_pop',
       'Município_x', 'Situação do domicílio', 'Grupo de idade', 'White',
       'Black', 'Brown', 'Indigenous', 'Município_y', 'population_density',
       'Município_x', 'Total', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'no_religion',
       'Município_y', 'Classes de rendimento nominal mensal', '<1', '1-3',
       '4-7', '8-10', '11-14', '>15', 'NA', 'Município', 'Destino do lixo',
       'house_income_BRL'],
      dtype='object')

In [ ]:
df2 = df_merged[['id_municipio', 'elderly(>70yo)','rural_pop', 'White', 'Black', 'Brown', 'Indigenous',
                 'population_density', 'Catholic', 'Evangelic', 'Pentecostal Evangelic', 'Umbanda/Candomble', 'no_religion',
                 '<1', '1-3', '4-7', '8-10', '11-14', '>15', 'house_income_BRL']]

In [ ]:
df2['id_municipio'] = df2['id_municipio'].astype(str)
df2.info()

<ipython-input-70-c802e040cb43>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id_municipio'] = df2['id_municipio'].astype(str)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5507 entries, 0 to 5506
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_municipio           5507 non-null   object 
 1   elderly(>70yo)         5507 non-null   float64
 2   rural_pop              5507 non-null   object 
 3   White                  5507 non-null   float64
 4   Black                  5507 non-null   object 
 5   Brown                  5507 non-null   float64
 6   Indigenous             5507 non-null   object 
 7   population_density     5507 non-null   float64
 8   Catholic               5507 non-null   int64  
 9   Evangelic              5507 non-null   object 
 10  Pentecostal Evangelic  5507 non-null   object 
 11  Umbanda/Candomble      5507 non-null   object 
 12  no_religion            5507 non-null   object 
 13  <1                     5507 non-null   float64
 14  1-3                    5507 non-null   float64
 15  4-7 

In [ ]:
df2.head(5)

,id_municipio,elderly(>70yo),rural_pop,White,Black,Brown,Indigenous,population_density,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,no_religion,<1,1-3,4-7,8-10,11-14,>15,house_income_BRL
0,1100015,3.76,42.73,43.39,4.49,49.70,2.04,3.45,14464,2574,3729,-,2726,13.79,27.30,40.14,10.13,6.52,0.74,1310.78
1,1100023,2.60,15.30,35.44,8.64,53.68,0.11,20.41,36570,6082,20004,-,18184,11.32,20.09,40.11,15.57,10.28,1.81,1849.30
2,1100031,4.13,57.34,46.22,7.37,44.78,0.04,4.80,3857,305,1462,-,575,14.07,24.92,43.54,9.87,5.25,0.29,1232.03
3,1100049,3.48,21.19,41.79,5.77,49.27,1.87,20.72,33535,11320,15970,-,10156,11.51,19.50,39.99,13.75,11.12,1.97,1853.64
4,1100056,4.18,15.33,47.83,5.08,46.07,0.08,6.12,10100,1624,2908,-,1458,11.52,21.62,43.38,12.36,8.83,0.64,1573.62


In [ ]:
df = df.merge(df2, on='id_municipio')
df.head(5)

,id_municipio,id_municipio_tse,nome,sigla_uf,nome_regiao,populacao,dose1_applied,dose2_applied,dose3_applied,dose4_applied,...,Pentecostal Evangelic,Umbanda/Candomble,no_religion,<1,1-3,4-7,8-10,11-14,>15,house_income_BRL
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,17525,14748,2325,6,...,3729,-,2726,13.79,27.30,40.14,10.13,6.52,0.74,1310.78
1,1100023,78,Ariquemes,RO,Norte,111148,73445,58687,12299,355,...,20004,-,18184,11.32,20.09,40.11,15.57,10.28,1.81,1849.30
2,1100031,450,Cabixi,RO,Norte,5067,4466,3736,580,45,...,1462,-,575,14.07,24.92,43.54,9.87,5.25,0.29,1232.03
3,1100049,94,Cacoal,RO,Norte,86416,73031,54926,9317,118,...,15970,-,10156,11.51,19.50,39.99,13.75,11.12,1.97,1853.64
4,1100056,272,Cerejeiras,RO,Norte,16088,12576,10441,2344,28,...,2908,-,1458,11.52,21.62,43.38,12.36,8.83,0.64,1573.62


In [ ]:
df['education_7'] = df[['<1', '1-3', '4-7']].sum(axis=1)

In [ ]:
df = df.rename(columns={'nome': 'Name', 'sigla_uf':'State', 'nome_regiao':'Region', 'populacao':'Population',
                        'dose1_applied':'Dose1 Applied', 'dose2_applied':'Dose2 Applied', 'dose3_applied':'Dose3 Applied',
                        'dose4_applied':'Dose4 Applied', 'dose1_received':'Dose1 Received', 'dose2_received':'Dose2 Received',
                        'dose3_received':'Dose3 Received', 'dose4_received':'Dose4 Received', 'doses_applied':'Doses Applied',
                        'doses_received':'Doses Received', 'Bolsonaro_share_2018':'Bolsonaro Share 18',
                        'Bolsonaro_share_2022':'Bolsonaro Share 22', 'far_left_2018':'Far-left 18', 'far_right_2018':'Far-right18',
                        'votos_psl_2018':'PSL coligation 18', 'far_left_2022':'Far-left 22', 'far_right_2022':'Far-right 22',
                        'votos_pl_2022':'PL coligation 22', 'elderly(70plus)':'Elderly (70yo plus)' ,'rural_pop': 'Rural pop.','population_density':'Pop. density','no_religion':'No religion',
                        'house_income_BRL':'Income (BRL)', 'education_7':'Education 8y less','11-14':'Education 11-14y', '8-10':'Education 8-10y',
                        '>15':'Education 15y plus', 'GeneralVaccination':'Vaccination coverage', 'BCG_coverage':'BCG coverage', 'Polio_coverage':'Polio coverage'})

In [ ]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Far-left 22', 'Far-right 22',
       'PL coligation 22', 'Vaccination coverage', 'BCG coverage',
       'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black',
       'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion', '<1',
       '1-3', '4-7', 'Education 8-10y', 'Education 11-14y',
       'Education 15y plus', 'Income (BRL)', 'Education 8y less'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5507 entries, 0 to 5506
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_municipio           5507 non-null   object 
 1   id_municipio_tse       5507 non-null   object 
 2   Name                   5507 non-null   object 
 3   State                  5507 non-null   object 
 4   Region                 5507 non-null   object 
 5   Population             5507 non-null   Int64  
 6   Dose1 Applied          5507 non-null   Int64  
 7   Dose2 Applied          5507 non-null   Int64  
 8   Dose3 Applied          5507 non-null   Int64  
 9   Dose4 Applied          5507 non-null   Int64  
 10  Dose1 Received         5507 non-null   Int64  
 11  Dose2 Received         5507 non-null   Int64  
 12  Dose3 Received         5507 non-null   Int64  
 13  Dose4 Received         5507 non-null   Int64  
 14  Doses Applied          5507 non-null   Int64  
 15  Dose

In [ ]:
df = df.drop(columns=['<1', '1-3', '4-7'])

df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Far-left 22', 'Far-right 22',
       'PL coligation 22', 'Vaccination coverage', 'BCG coverage',
       'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black',
       'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)', 'Education 8y less'],
      dtype='object')

In [ ]:
columns = ['Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied','Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
           'Dose4 Received', 'Doses Applied', 'Doses Received','Catholic', 'Evangelic', 'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion']

df[columns] = df[columns].replace('-', 0).astype('int64')

for column in columns:
  df[column] = df[column] / df['Population']

In [ ]:
df.head(10)

,id_municipio,id_municipio_tse,Name,State,Region,Population,Dose1 Applied,Dose2 Applied,Dose3 Applied,Dose4 Applied,...,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,No religion,Education 8-10y,Education 11-14y,Education 15y plus,Income (BRL),Education 8y less
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,0.778335,0.655001,0.10326,0.000266,...,0.642388,0.114319,0.165616,0.0,0.121069,10.13,6.52,0.74,1310.78,81.23
1,1100023,78,Ariquemes,RO,Norte,111148,0.660786,0.528008,0.110654,0.003194,...,0.329021,0.05472,0.179976,0.0,0.163602,15.57,10.28,1.81,1849.30,71.52
2,1100031,450,Cabixi,RO,Norte,5067,0.881389,0.73732,0.114466,0.008881,...,0.7612,0.060193,0.288534,0.0,0.113479,9.87,5.25,0.29,1232.03,82.53
3,1100049,94,Cacoal,RO,Norte,86416,0.84511,0.6356,0.107816,0.001365,...,0.388065,0.130994,0.184804,0.0,0.117525,13.75,11.12,1.97,1853.64,71.00
4,1100056,272,Cerejeiras,RO,Norte,16088,0.781701,0.648993,0.145699,0.00174,...,0.627797,0.100945,0.180756,0.0,0.090627,12.36,8.83,0.64,1573.62,76.52
5,1100064,230,Colorado do Oeste,RO,Norte,15213,0.859265,0.678104,0.112798,0.001775,...,0.704792,0.058437,0.25419,0.0,0.098666,12.94,10.23,1.32,1518.89,73.01
6,1100072,655,Corumbiara,RO,Norte,7052,0.832388,0.724759,0.155559,0.000425,...,0.82388,0.057856,0.233976,0.0,0.106069,5.95,2.95,0.34,1188.16,89.41
7,1100080,213,Costa Marques,RO,Norte,19255,0.461698,0.368424,0.060608,0.000675,...,0.448195,0.061023,0.123448,0.000208,0.034329,9.72,6.54,1.31,1225.56,81.35
8,1100098,256,Espigão D'Oeste,RO,Norte,33009,0.69169,0.530128,0.053379,0.000515,...,0.382805,0.208004,0.12954,0.0,0.106304,10.91,6.00,0.73,1455.66,79.78
9,1100106,19,Guajará-Mirim,RO,Norte,46930,0.634626,0.43974,0.041168,0.000384,...,0.512849,0.093394,0.132325,0.000192,0.083273,17.19,12.16,2.32,1646.55,64.65


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5507 entries, 0 to 5506
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_municipio           5507 non-null   object 
 1   id_municipio_tse       5507 non-null   object 
 2   Name                   5507 non-null   object 
 3   State                  5507 non-null   object 
 4   Region                 5507 non-null   object 
 5   Population             5507 non-null   Int64  
 6   Dose1 Applied          5507 non-null   Float64
 7   Dose2 Applied          5507 non-null   Float64
 8   Dose3 Applied          5507 non-null   Float64
 9   Dose4 Applied          5507 non-null   Float64
 10  Dose1 Received         5507 non-null   Float64
 11  Dose2 Received         5507 non-null   Float64
 12  Dose3 Received         5507 non-null   Float64
 13  Dose4 Received         5507 non-null   Float64
 14  Doses Applied          5507 non-null   Float64
 15  Dose

In [ ]:
df.to_csv('df.csv', index=False)